<a href="https://colab.research.google.com/github/StatPhysBio/protein_holography-web/blob/main/HCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <b>Install condacolab

try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

✨🍰✨ Everything looks OK!


In [ ]:
#@title <b>Download repository, models and setup environment

#@markdown If prompted to restart the runtime due to some packages being already installed, simply restart and re-run the cells above and including this one. The conflict should resolve itself after 1 or 2 restarting prompts. We are currently working on making that not happen.

! rm -r sample_data

! pip install torch==1.13.1
! conda install -c conda-forge openmm==8.0.0
! conda install -c conda-forge pdbfixer==1.9
! pip install pandas==1.5.0
! pip install e3nn==0.5.0

! git clone https://github.com/StatPhysBio/protein_holography-web.git
%cd protein_holography-web
! git clone https://github.com/StatPhysBio/zernikegrams.git
%cd zernikegrams
! pip install .
%cd ..
! pip install .
%cd ..




rm: cannot remove 'sample_data': No such file or directory
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

  Using cached e3nn-0.5.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached scipy-1.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached opt_einsum_fx-0.1.4-py3-none-any.whl.metadata (3.3 kB)
  Usin

In [ ]:
#@title <b>Single PDB request</b>

#@markdown Choose between <b>ONE</font></b> of the possible input sources for the target pdb.
#@markdown - AlphaFold2 PDB (v4) via Uniprot ID:
AF_ID =''#@param {type:"string"}
#@markdown - PDB ID (imported from RCSB PDB):
PDB_ID ='1AO7'#@param {type:"string"}
#@markdown - Specify path to pdb
PDB_PATH ='' #@param {type:"string"}

#@markdown Select target chain. Default is "all" which computes all chains. Note that we do not remove the other chains from the PDB file.
CHAIN_ID='B' #@param {type:'string'}



# download pdbs
import os

TEMP_PDB_DIR = "/content/pdb_dir/"
TEMP_PDB_AND_CHAIN_FILE = '/content/pdbs_and_chains.txt'

if os.path.exists(TEMP_PDB_DIR):
    os.system(f'rm -r {TEMP_PDB_DIR}')
os.makedirs(TEMP_PDB_DIR)

if os.path.exists(TEMP_PDB_AND_CHAIN_FILE):
    os.system(f'rm {TEMP_PDB_AND_CHAIN_FILE}')


class TooManyInputsError(Exception):
    pass

if PDB_PATH != '':
    if AF_ID != '' or PDB_ID != '': raise TooManyInputsError('Please specify only one between AF_ID, PDB_ID, and PDB_PATH')
    os.system(f"cp {PDB_PATH} {TEMP_PDB_DIR}")
    filename = '.'.join(os.basename(PDB_PATH).split('.')[:-1])

elif AF_ID !='':
    if len(AF_ID) < 6: raise ValueError("AF_ID must have length at least 6 since it's a uniprot ID")
    if PDB_PATH != '' or PDB_ID != '': raise TooManyInputsError('Please pecify only one between AF_ID, PDB_ID, and PDB_PATH')
    os.system(f'curl -s -f https://alphafold.ebi.ac.uk/files/AF-{AF_ID}-F1-model_v4.pdb -o {os.path.join(TEMP_PDB_DIR, f"AF-{AF_ID}-F1-model_v4.pdb")}')
    filename = f'AF-{AF_ID}-F1-model_v4'

elif PDB_ID !='':
    if len(PDB_ID) != 4: raise ValueError("PDB_ID must have length 4 to be valid.")
    if PDB_PATH != '' or AF_ID != '': raise TooManyInputsError('Please pecify only one between AF_ID, PDB_ID, and PDB_PATH')
    os.system(f'curl -s -f https://files.rcsb.org/download/{PDB_ID}.pdb -o {os.path.join(TEMP_PDB_DIR, f"{PDB_ID}.pdb")}')
    filename = PDB_ID

else:
    raise ValueError('Please specify one of the above inputs.')

if len(CHAIN_ID) > 1 and CHAIN_ID != 'all':
    raise ValueError(f'CHAIN_ID must be a single characher, cannot be "{CHAIN_ID}"')


with open(TEMP_PDB_AND_CHAIN_FILE, 'w+') as f:
    f.write(filename)
    if CHAIN_ID != 'all':
        f.write(' ' + CHAIN_ID)



In [ ]:
#@title <b>Multiple PDBs request</b>

## TODO

In [ ]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
#@title <b>Run selected model with desired outputs</font></b>

#@markdown <b>Choose one and only one model version<b>
HCNN_biopython_proteinnet_0p00=True#@param {type:"boolean"}
HCNN_biopython_proteinnet_0p50=False#@param {type:"boolean"}
HCNN_biopython_proteinnet_extra_mols_0p00=False#@param {type:"boolean"}
HCNN_biopython_proteinnet_extra_mols_0p50=False#@param {type:"boolean"}

if HCNN_biopython_proteinnet_0p00:
    if HCNN_biopython_proteinnet_0p50 or HCNN_biopython_proteinnet_extra_mols_0p00 or HCNN_biopython_proteinnet_extra_mols_0p50: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_0p00"
elif HCNN_biopython_proteinnet_0p50:
    if HCNN_biopython_proteinnet_0p00 or HCNN_biopython_proteinnet_extra_mols_0p00 or HCNN_biopython_proteinnet_extra_mols_0p50: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_0p50"
elif HCNN_biopython_proteinnet_extra_mols_0p00:
    if HCNN_biopython_proteinnet_0p00 or HCNN_biopython_proteinnet_0p50 or HCNN_biopython_proteinnet_extra_mols_0p50: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_extra_mols_0p00"
elif HCNN_biopython_proteinnet_extra_mols_0p50:
    if HCNN_biopython_proteinnet_0p00 or HCNN_biopython_proteinnet_0p50 or HCNN_biopython_proteinnet_extra_mols_0p00: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_extra_mols_0p50"
else:
    raise ValueError("Please select one and only one model.")

#@markdown The first two models exclude extra ligands and ions in the PDB files.
#@markdown The other two keep them. All models remove waters. \
#@markdown "_0p00" and "_0p50" denote the amount of noise - in Angstrom - with which the models were trained.

#@markdown

#@markdown <b>Choose desired outputs<b>
probas=True#@param {type:"boolean"}
logprobas=False#@param {type:"boolean"}
logits=False#@param {type:"boolean"}
embeddings=False#@param {type:"boolean"}

REQUEST_STRING = ''
if probas: REQUEST_STRING += ' probas'
if logprobas: REQUEST_STRING += ' logprobas'
if logits: REQUEST_STRING += ' logits'
if embeddings: REQUEST_STRING += ' embeddings'
REQUEST_STRING = REQUEST_STRING.strip()

#@markdown Probas, logprobas and/or logits will be saved in file `output_{MODEL_VERSION}.csv`. \
#@markdown Embeddings will be saved in file `output_{MODEL_VERSION}-embeddings.npy`.


os.system(f"python protein_holography-web/run_hcnn_on_pdbfiles.py \
            -m {MODEL_VERSION} \
            -pd {TEMP_PDB_DIR} \
            -pn {TEMP_PDB_AND_CHAIN_FILE} \
            -o output_{MODEL_VERSION}.csv \
            -v 0 \
            -lb 1 \
            -r {REQUEST_STRING}")



0